# Nhóm 16 - Tối ưu hóa
## Đề tài: Sử dụng thuật toán Monte-Carlo Tree Search vào việc tối ưu hóa nước đi và chiến lược khi chơi cờ vây.
### Các thành viên:
### - Nguyễn Công Trường (Nhóm trưởng) - 22000129 - K67A2 Toán tin.
### - Dương Đạt Khang - 22000099 - K67A2 Toán tin.
### - Trịnh Đức Huy - 22000097 - K67A2 Toán tin.

### Tài liệu tham khảo:  
### - Sylvain Gelly, Marc Schoenauer, Michèle Sebag, Olivier Teytau,  Levente Kocsis, David Silver, Csaba Szepesvári - The Grand Challenge of Computer Go: Monte Carlo Tree Search and Extensions (2012).
### -  Adrien Couetoux, Martin Muller, Olivier Teytaud - Monte Carlo Tree Search in Go (2013).

### Code tham khảo: 
### https://github.com/nikzaugg/Go-Game (thuật toán, triển khai trò chơi).
### https://github.com/MihailoMilenkovic/GoMCTS (giao diện, agent).


In [1]:
#!/usr/bin/env python2.7
# -*- coding: utf-8 -*-
"""
Mô-đun này chứa giải pháp ví dụ cho việc tìm kiếm, đánh dấu lãnh thổ
và đếm điểm số và một lớp đại diện cho một nhóm đá.

Mô hình trò chơi cần kế thừa từ Terr_Template nếu nó không
tự triển khai các phương thức đó.
"""

class Terr_Template(object):
    """Lớp này không hoạt động độc lập mà có thể được kế thừa bởi
    Model."""
    
    def find_territory(self):
        """Cố gắng tự động đánh dấu lãnh thổ cho người chơi phù hợp.

        Thuật toán hiện tại:
            Nó chỉ đánh dấu các khu vực trống hoàn toàn bị bao quanh
            bởi một màu.
            Do đó, nó sẽ không nhận ra tù nhân hoặc các nhóm đã chết.

        Các thuộc tính được cập nhật bởi hàm này:
            self.score
            self.territory
        """

        # Theo dõi các ô đã được kiểm tra
        area = []
        for y in range(self.size):
            for x in range(self.size):

                if (x, y) in area:
                    continue

                # Chỉ xem các ô trống
                if self.board[y][x] is None:
                    # Tìm tất cả các ô trống liền kề
                    # Count chứa số lượng quân liền kề
                    # của mỗi màu.
                    _a, count = self._find_empty(x, y, area=area)
                    area += _a

                    # Đánh dấu lãnh thổ nếu một màu không có
                    # quân cờ liền kề
                    if count[BLACK] == 0 and count[WHITE] > 0:
                        self._claim_empty(x, y, WHITE)
                    elif count[WHITE] == 0 and count[BLACK] > 0:
                        self._claim_empty(x, y, BLACK)

        # Tính lại điểm số
        self._compute_score()

    def mark_territory(self, x, y):
        """Hàm có thể được gọi bởi người dùng để đánh dấu lãnh thổ cho
        một người chơi.
        Đối với các ô trống, nó cũng sẽ đánh dấu tất cả các ô trống liền kề,
        đối với các ô chứa quân cờ, nó sẽ đánh dấu toàn bộ nhóm quân cờ
        và tất cả các ô trống liền kề.

        Tham số:
            x, y (int): tọa độ của ô

        Các thuộc tính được cập nhật bởi hàm này:
            self.score
            self.territory
        """

        
        if not self.game_over:
            return

        # Đánh dấu một ô trống
        if self.board[y][x] is None:
            # Chuyển đổi qua các màu tùy thuộc vào cách
            # ô hiện đang được đánh dấu
            col_dict = {None:BLACK, BLACK:WHITE, WHITE:None}
            color = col_dict[self.territory[y][x]]
            # Bắt đầu đánh dấu đệ quy các ô
            self._claim_empty(x, y, color)

        # Đánh dấu một nhóm
        else:
            # Chọn đánh dấu hoặc bỏ đánh dấu nhóm
            if self.territory[y][x] is None:
                color = not self.board[y][x].color
            else:
                color = None
            # Bắt đầu đánh dấu đệ quy các ô
            self._claim_group(x, y, color)

        # Tính lại điểm số
        self._compute_score()

        def _claim_empty(self, x, y, color, area=None):
            """Phần đầu tiên của hàm đệ quy, đánh dấu một ô trống và sau đó cố gắng đánh dấu tất cả các ô liền kề.

            Tham số:
                x, y (int)       : tọa độ
                color (bool/None): Màu mà lãnh thổ sẽ được đánh dấu.
                area (list)      : Để đệ quy theo dõi các ô đã được kiểm tra.

            Các thuộc tính được cập nhật bởi hàm này:
                self.territory
            """

            # Không làm điều đó trong định nghĩa hàm, vì khi đó bạn sẽ gặp
            # hành vi kỳ lạ (đối tượng có thể thay đổi) và area sẽ không trống
            # khi bạn mong đợi nó trống.
            if area is None:
                area = []

            # Dừng đệ quy
            if self.board[y][x] is not None or (x, y) in area:
                return

            # Đánh dấu ô
            area.append((x, y))
            self.territory[y][x] = color

            # Đệ quy qua tất cả các ô liền kề
            for (u, v) in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:

                # Nếu không nằm trên bảng
                if u < 0 or v < 0 or u >= self.size or v >= self.size:
                    continue

                # Chỉ đánh dấu các ô trống liền kề
                if (u, v) not in area and self.board[v][u] is None:
                    self._claim_empty(u, v, color, area=area)

    def _claim_empty(self, x, y, color, area=None):
        """Phần đầu tiên của hàm đệ quy, đánh dấu một ô trống và sau đó cố gắng đánh dấu tất cả các ô liền kề.

        Tham số:
            x, y (int)       : tọa độ
            color (bool/None): Màu mà lãnh thổ sẽ được đánh dấu.
            area (list)      : Để đệ quy theo dõi các ô đã được kiểm tra.

        Các thuộc tính được cập nhật bởi hàm này:
            self.territory
        """

        # Không làm điều đó trong định nghĩa hàm, vì khi đó bạn sẽ gặp
        # hành vi kỳ lạ (đối tượng có thể thay đổi) và area sẽ không trống
        # khi bạn mong đợi nó trống.
        if area is None:
            area = []

        # Dừng đệ quy
        if self.board[y][x] is not None or (x, y) in area:
            return

        # Đánh dấu ô
        area.append((x, y))
        self.territory[y][x] = color

        # Đệ quy qua tất cả các ô liền kề
        for (u, v) in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:

            # Nếu không nằm trên bảng
            if u < 0 or v < 0 or u >= self.size or v >= self.size:
                continue

            # Chỉ đánh dấu các ô trống liền kề
            if (u, v) not in area and self.board[v][u] is None:
                self._claim_empty(u, v, color, area=area)

    def _claim_group(self, x, y, color, area=None):
        """Phần thứ hai của hàm đệ quy, đánh dấu một nhóm cho đội đối thủ
        (các quân cờ chết) và cũng đánh dấu tất cả các ô trống liền kề.

        Tham số:
            x, y (int)       : tọa độ
            color (bool/None): Màu mà lãnh thổ sẽ được đánh dấu.
            area (list)      : Để đệ quy theo dõi các ô đã được kiểm tra.

        Các thuộc tính được cập nhật bởi hàm này:
            self.territory
        """
        if area is None:
            area = []
        # Đánh dấu nhóm
        for u, v in self.board[y][x].stones:
            if (u, v) not in area:
                area.append((u, v))
                self.territory[v][u] = color
        # Đệ quy qua tất cả các ô liền kề
        # (trong biên của nhóm)
        for u, v in self.board[y][x].border:
            # Đánh dấu tất cả các ô trống liền kề
            if self.board[v][u] is None and (u, v) not in area:
                self._claim_empty(u, v, color, area=area)

    def _compute_score(self):
        """Tính điểm của người chơi từ lãnh thổ hiện tại và số quân cờ bị bắt.
        Thiết lập:
            self.score (list): tổng số quân cờ bị bắt và số ô lãnh thổ được đánh dấu màu
        """
        self.score = [0, 0]
        for j in range(self.size):
            for i in range(self.size):
                # Đếm lãnh thổ đen
                if self.territory[j][i] == BLACK:
                    self.score[BLACK] += 1
                    # Điểm bổ sung cho các quân cờ chết bên trong lãnh thổ
                    if self.board[j][i] is not None:
                        self.score[BLACK] += 1
                # Đếm lãnh thổ trắng
                elif self.territory[j][i] == WHITE:
                    self.score[WHITE] += 1
                    # Điểm bổ sung cho các quân cờ chết bên trong lãnh thổ
                    if self.board[j][i] is not None:
                        self.score[WHITE] += 1

    def _find_empty(self, x, y, area=None, count=None):
        """Tìm tất cả các ô trống liền kề bắt đầu từ (x, y) và
        đếm số lượng quân cờ liền kề của mỗi màu.

        Trả về:
            area (list) : Danh sách các ô trống
            count (list): Số lượng quân cờ liền kề của [đen, trắng]

        """
        if area is None:
            area = []
        if count is None:
            count = [0, 0]
        # Dừng đệ quy
        if self.board[y][x] is not None or (x, y) in area:
            return area, count
        area.append((x, y))
        # Liền kề
        for (u, v) in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            # Nếu không nằm trong bảng
            if u < 0 or v < 0 or u >= self.size or v >= self.size:
                continue
            # Đi qua các ô trống một cách đệ quy và thêm các quân cờ liền kề
            if (u, v) not in area:
                if self.board[v][u] is None:
                    self._find_empty(u, v, area=area, count=count)
                else:
                    count[self.board[v][u].color] += 1
        return area, count


class Grp_Template(object):
    def _kill(self, grp):
        """Loại bỏ một nhóm quân cờ khỏi trò chơi và tăng bộ đếm số quân cờ bị bắt.

        Tham số:
            grp (Group): Nhóm quân cờ cần được loại bỏ

        Các thuộc tính được cập nhật bởi hàm này:
            self.board
            self.captured
            self.group
        """
        self.captured[not grp.color] += grp.size
        self._remove(grp)

    def _liberties(self, grp):
        """Đếm số lượng ô trống liền kề với nhóm.

        Tham số:
            grp (Group): Một nhóm quân cờ.

        Trả về:
            (int): số lượng tự do của nhóm đó


        """
        return sum([1 for u, v in grp.border if self.board[v][u] is None])


class Group(object):
    """Đại diện cho một nhóm các quân cờ kết nối trên bàn cờ.

    Thuộc tính:
        stones (set): danh sách tất cả các tọa độ nơi nhóm có quân cờ
        border (set): danh sách tất cả các ô liền kề với nhóm
                      Đối với một nhóm mới, các ô trống phải được thêm thủ công
                      vì nhóm không biết về kích thước bàn cờ
        color (bool): màu của nhóm

    Property:
        size (int): bằng với len(self.stones), số lượng quân cờ trong
                    nhóm.
    """

    def __init__(self, stones=None, color=None):
        """
        Khởi tạo nhóm
        """
        if stones is not None:
            self.stones = set(stones)
        else:
            self.stones = set()

        self.border = set()
        self.color = color

    def __add__(self, other):
        """Thêm hai nhóm cùng màu
        Nhóm mới chứa tất cả các quân cờ của các nhóm trước đó và
        biên sẽ được cập nhật đúng cách.

        Raises:
            TypeError: Màu sắc của các nhóm không khớp
        """
        if self.color != other.color:
            raise ValueError('Only groups of same colour can be added!')
        grp = Group(stones=self.stones.union(other.stones))
        grp.color = self.color
        grp.border = self.border.union(other.border).difference(grp.stones)
        return grp

    @property
    def size(self):
        """Kích thước của nhóm"""
        return len(self.stones)



In [2]:
#!/usr/bin/env python2.7
# -*- coding: utf-8 -*-

# Author: Moritz Eck

# Model part of the MVC architecture for an implementation of the Go game

#from template import Group
from copy import deepcopy
# constants
BLACK = True
WHITE = False

class Model(object):
    """ 
    Lớp này chịu trách nhiệm thực hiện tất cả các tính toán và logic của trò chơi.
    """
    def __init__(self, n=11):
        """Hàm này khởi tạo một mô hình mới.

        Tham số:
            n (int) - kích thước của lưới

        Các thuộc tính được khởi tạo bởi hàm này:
            self.size
            self.turn
            self.blocked_field
            self.has_passed
            self.game_over
            self.board
            self.territory
            self.score
            self.captured
        """
        # thuộc tính trò chơi
        self.size = n
        self.turn = BLACK

        # được sử dụng cho luật ko
        self.blocked_field = None

        # nhận diện người chơi bỏ lượt
        self.has_passed = False

        # nhận diện trò chơi kết thúc
        self.game_over = False

        # bảng được biểu diễn bởi ma trận nxn.
        self.board = [[None for i in range(n)] for j in range(n)]   
        self.territory = [[None for i in range(n)] for j in range(n)]

        # điểm số từ các ô trống vào cuối trò chơi.
        self.score = [0, 0]

        # số quân cờ bị bắt trong trò chơi
        self.captured = [0, 0] 

    def printBoard(self):
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if(self.board[i][j]==None):
                    print("_",end="")
                else:
                    if self.board[i][j].color:
                        print("#",end="")
                    else:
                        print("O",end="")
            print()
    
    def copy(self):
        ans=Model(self.size)
        ans.size = self.size
        ans.turn = self.turn
        ans.blocked_field = self.blocked_field
        ans.has_passed = self.has_passed
        ans.game_over = self.game_over
        ans.board=deepcopy(self.board)
        ans.territory=deepcopy(self.territory)
        ans.score=[self.score[i] for i in range(len(self.score))]
        ans.captured=[self.captured[i] for i in range(len(self.captured))]
        return ans

    def getFinalScores(self):
        assert(self.game_over)
        numBlackStones=0
        numWhiteStones=0
        for i in range(len(self.board)):
            for j in range(len(self.board)):
                if self.board[i][j]!=None:
                    if self.board[i][j].color:
                        numBlackStones+=1
                    else:
                        numWhiteStones+=1

        firstPlayerReward=self.score[0]+numWhiteStones
        secondPlayerReward=self.score[1]+numBlackStones
        return [firstPlayerReward,secondPlayerReward]

    def getReward(self):
        '''Hàm trả về "phần thưởng" của trạng thái trò chơi (giả sử trò chơi đã kết thúc)'''
        score=self.getFinalScores()
        if score[0]>score[1]:
            return -1
        elif score[0]==score[1]:
            return 0
        else:
            return 1

    def passing(self):
        """Hành động khi một người chơi bỏ lượt.

        Các biến được thay đổi bởi hàm này:
            self.game_over
            self.turn
            self.has_passed
            self.blocked_field
        """

        # không làm gì nếu trò chơi kết thúc
        if self.game_over:
            return False
        
        # nếu cả hai người chơi đều bỏ lượt thì trò chơi kết thúc
        if self.has_passed:
            self.game_over = True
            return True
        
        # đổi lượt và đặt has_passed thành True
        self.turn = WHITE if (self.turn == BLACK) else BLACK     
        self.has_passed = True
        self.blocked_field = None

        return True

    def _stones(self):
        """Trả về một danh sách lồng nhau (cùng hình dạng với bảng) chứa màu của mỗi quân cờ.

        Trả về:
            list (boolean) : danh sách đa chiều chứa màu của các quân cờ trên bảng.
        """
        # khởi tạo một danh sách rỗng mới với kích thước của bảng chơi
        colors = [[None for i in range(self.size)] for j in range(self.size)]

        for j in range(0, self.size):
            for i in range(0, self.size):
                if self.board[j][i] is None:
                    colors[j][i] = None
                else:
                    colors[j][i] = self.board[j][i].color
                            
        return colors

    def _add(self, grp):
        """Thêm một nhóm quân cờ vào trò chơi

        Tham số:
            grp (Group): Nhóm quân cờ sẽ được thêm vào
        
        Các thuộc tính được cập nhật bởi hàm này:
            self.board
        """
        for (x, y) in grp.stones:
            self.board[y][x] = grp
    
    def _remove(self, grp):
        """Loại bỏ một nhóm quân cờ khỏi trò chơi

        Tham số:
            grp (Group): Nhóm quân cờ sẽ bị loại bỏ
        
        Các thuộc tính được cập nhật bởi hàm này:
            self.board
        """
        for (x, y) in grp.stones:
            self.board[y][x] = None

    def _kill(self, grp):
        """Loại bỏ một nhóm quân cờ khỏi trò chơi và tăng bộ đếm số quân cờ bị bắt.

        Tham số:
            grp (Group): Nhóm quân cờ đã bị bắt - cần phải loại bỏ

        Các thuộc tính được cập nhật bởi hàm này:
            self.board
            self.captured
            self.group
        """
        # tăng bộ đếm số quân cờ bị bắt của màu đối diện bằng số quân cờ trong nhóm
        self.captured[not grp.color] += grp.size

        # loại bỏ nhóm quân cờ
        self._remove(grp)

    def _liberties(self, grp):
        """Đếm số lượng ô trống liền kề với nhóm.

        Tham số:
            grp (Group): Nhóm quân cờ cần được kiểm tra

        Trả về:
            int: số lượng ô trống liền kề với nhóm
        """
        return sum([1 for u, v in grp.border if self.board[v][u] is None])     

    def add_scores(self):
        """Tổng hợp điểm số: cộng các ô trống + số quân cờ bị bắt của mỗi người chơi

          Trả về:
            list (int): chứa điểm số của mỗi người chơi
        """
        return [self.score[0] + self.captured[0], self.score[1] + self.captured[1]]

    def get_data(self):
        """Trả về đối tượng dữ liệu chứa tất cả thông tin liên quan đến bộ điều khiển.
          Trả về:

            data (dictionary): đối tượng dữ liệu cho bộ điều khiển
        """
        data = {
            'size'      : self.size,
            'stones'    : self._stones(),
            'territory' : self.territory,
            'game_over' : self.game_over,
            'score'     : self.add_scores(),
            'color'     : self.turn
        }

        return data

    def legal_move(self,x,y):
        """Kiếm tra xem nước đi có hợp lệ"""
        tableCopy=self.copy()
        return tableCopy.place_stone(x,y)

    def place_stone(self, x, y):
        """Cố gắng đặt một quân cờ mới.
           Xác thực nước đi và nếu hợp lệ, thực hiện hành động tương ứng.
        Tham số:
            x (int): tọa độ x của quân cờ mới
            y (int): tọa độ y của quân cờ mới
        Các thuộc tính được cập nhật bởi hàm này:
            self.board - thêm / loại bỏ / giết quân cờ
        """
        # không thể đặt quân cờ nếu trò chơi đã kết thúc
        if self.game_over:
            return False
        
        # kiểm tra nếu người chơi bỏ lượt
        if x==-1 and y==-1:
            return self.passing()

        # kiểm tra xem vị trí có trống không
        if self.board[y][x] is not None:
            return False

        # không thể đặt quân cờ trên ô bị chặn
        if self.blocked_field == (x, y):
            return False

        # tạo nhóm mới với tọa độ đã cho
        new_group = Group(stones=[(x, y)], color=self.turn)

        # tạo hai danh sách để ghi nhớ các nhóm cần loại bỏ / giết
        groups_to_remove = []
        groups_to_kill = []

        ######################################
        # Xác thực nước đi
        ######################################

        # đặt tính hợp lệ của nước đi ban đầu là False
        is_valid = False

        # kiểm tra các ô liền kề
        for (u, v) in [(x-1, y), (x+1, y), (x, y-1), (x, y+1)]:
            if u < 0 or v < 0 or u >= self.size or v >= self.size:
                continue
            
            # Thêm ô liền kề vào biên của nhóm mới
            new_group.border.add((u, v))

            other_group = self.board[v][u]
            
            # kiểm tra nếu ô liền kề là None
            if other_group is None:
                is_valid = True
                continue

            # nếu cùng màu quân thì cho vào chung 1 nhóm
            if new_group.color == other_group.color:
                new_group = new_group + other_group
                groups_to_remove.append(other_group)

            # các nhóm có màu khác nhau
            # kiểm tra rằng chỉ có một ô trống liền kề với nhóm khác
            elif self._liberties(other_group) == 1:
                is_valid = True
                if other_group not in groups_to_kill: 
                    groups_to_kill.append(other_group)

        # nhóm mới phải có ít nhất một ô trống liền kề
        if self._liberties(new_group) >= 1:
            is_valid = True

        ######################################
        # Xử lý nước đi
        ######################################

        # nếu nước đi hợp lệ
        if is_valid:
            # xóa các nhóm 
            for grp in groups_to_remove:
                self._remove(grp)

            # bắt các nhóm 
            for grp in groups_to_kill:
                self._kill(grp)

            # bổ sung nhóm mới
            self._add(new_group)
        
        # nếu không hợp lệ
        else:
            return False

        ######################################
        # luật ko: chặn ô vừa đặt quân cờ
        ######################################

        # có 3 điều kiện để áp dung luật ko:
        # 1. nhóm mới chỉ có một quân cờ
        # 2. mới chỉ có một nhóm bị bắt
        # 3. nhóm bị bắt cũng chỉ có một quân cờ
        if new_group.size == 1 and len(groups_to_kill) == 1 and groups_to_kill[0].size == 1:
            for (x, y) in groups_to_kill[0].stones:
                self.blocked_field = (x, y)
        else:
            self.blocked_field = None
        
        ######################################
        # Đổi lượt chơi
        ######################################
        self.turn = WHITE if (self.turn == BLACK) else BLACK
        self.has_passed = False

        return True

    def _compute_score(self):
        """Đếm số lượng ô đã được đánh dấu và cập nhật điểm số.

          Các biến được thay đổi bởi hàm này:
            self.score
        """

        # khởi tạo điểm là 0
        self.score = [0, 0]
        for j in range(0, self.size):
            for i in range(0, self.size):
                # đếm số quân đen
                if self.territory[j][i] == BLACK:
                    if self.board[j][i] != None:
                        # thêm 1 điểm bổ sung cho các quân cờ chết bên trong lãnh thổ 
                        self.score[BLACK] += 2
                    else:
                        self.score[BLACK] += 1
                        
                # đếm số quân trắng       
                elif self.territory[j][i] == WHITE:
                    if self.board[j][i] != None:
                        # thêm 1 điểm bổ sung cho các quân cờ chết bên trong lãnh thổ  
                        self.score[WHITE] += 2
                    else:
                        self.score[WHITE] += 1

    def _claim_empty(self, x, y, color, area=None):
        """ Yêu cầu một ô trống bao gồm tất cả các ô trống liền kề (lân cận).

        Tham số:
            x (int): tọa độ x
            y (int): tọa độ y
            color (boolean): màu của ô trống sẽ nhận
            area (list): các tọa độ / ô đã được duyệt
        
        Các biến được thay đổi bởi hàm này:
            self.territory
        """

        # khởi tạo một danh sách rỗng mới
        if area is None:
            area = list()

        # điều kiện dừng đệ quy
        # vị trí không trống hoặc đã được duyệt qua
        if self.board[y][x] is not None or (x, y) in area:
            return

        # đánh dấu ô trống
        self.territory[y][x] = color
        area.append((x, y))

        for (u, v) in [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]:
            if u < 0 or v < 0 or u >= self.size or v >= self.size:
                continue            
            if (u, v) not in area and self.board[v][u] is None:
                self._claim_empty(u, v, color, area=area)

    def _claim_group(self, x, y, color, area=None):
        """Yêu cầu một nhóm quân cờ bao gồm tất cả các ô trống liền kề.

        Tham số:
            x (int) - tọa độ x
            y (int) - tọa độ y
            color (boolean) - màu của nhóm
            area (list) - các tọa độ / ô đã được duyệt

        Các biến được thay đổi bởi hàm này:
            self.territory
        """
        if area is None:
            area = list()
        for (u, v) in self.board[y][x].stones:
            if (u, v) not in area:
                area.append((u, v))
                self.territory[v][u] = color
        for (u, v) in self.board[y][x].border:
            if self.board[v][u] is None and (u, v) not in area:
                self._claim_empty(u, v, color, area=area) 
    
    def _find_empty(self, x, y, area=None, count=None):
        """ Tìm tất cả các ô trống liền kề bắt đầu từ (x, y) và đếm số lượng quân cờ liền kề của mỗi màu.

        Trả về:
            area (list):    danh sách các ô trống
            count (list):   số lượng quân cờ liền kề của [đen, trắng]
        """
        if area is None:
            area = list()
        if count is None:
            count = [0, 0]
        if self.board[y][x] is not None or (x, y) in area:
            return area, count
        area.append((x, y))

        for (u, v) in [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]:
            if u < 0 or v < 0 or u >= self.size or v >= self.size:
                continue
            if (u, v) not in area:
                if self.board[v][u] is None:
                    self._find_empty(u, v, area=area, count=count)
                else:
                    count[self.board[v][u].color] += 1
        return area, count

    def mark_territory(self, x, y):
        """Đánh dấu lãnh thổ cho người chơi tương ứng.

        Tham số:
            x (int): tọa độ x 
            y (int): tọa độ y

        Các thuộc tính được cập nhật bởi hàm này:
            self.score
            self.territory
        """

        if not self.game_over:
            return

        # đánh dấu một ô trống
        if self.board[y][x] is None:
            # chuyển đổi qua các màu tùy thuộc vào cách ô hiện được đánh dấu
            # None => Đen => Trắng => None
            # None => Black => White => None
            col_dict = {None:BLACK, BLACK:WHITE, WHITE:None}

            # thay đổi màu
            color = col_dict[self.territory[y][x]]

            # đánh dấu ô trống
            self._claim_empty(x, y, color)
        else:
            if self.territory[y][x] is None:
                color = not self.board[y][x].color
            else:
                color = None
            self._claim_group(x, y, color)
        self._compute_score()

    def find_territory(self):
        """Cố gắng tự động đánh dấu lãnh thổ cho người chơi tương ứng.

        Nhược điểm:
            Nó chỉ đánh dấu các khu vực trống hoàn toàn bị bao quanh bởi một màu.
            Do đó, nó sẽ không nhận ra các tù nhân hoặc các nhóm đã chết.

        Các thuộc tính được cập nhật bởi hàm này:
            self.score
            self.territory
        """

        covered_area = list()

        for y in range(self.size):
            for x in range(self.size):
                if (x, y) in covered_area:
                    continue

                if self.board[y][x] is None:
                    area, count = self._find_empty(x, y, area=covered_area)
                    covered_area += area

                    if count[BLACK] == 0 and count[WHITE] > 0:
                        self._claim_empty(x, y, WHITE)

                    elif count[WHITE] == 0 and count[BLACK] > 0:
                        self._claim_empty(x, y, BLACK)

        self._compute_score()

In [3]:

'''
Lớp Model:
  các trường:
    self.size - kích thước bàn cờ
    self.turn - ĐEN hoặc TRẮNG
    self.blocked_field - một số boolean cho quy tắc ko, không rõ
    self.has_passed - để kiểm tra nếu nước đi cuối cùng là bỏ qua
    self.game_over - để kiểm tra nếu trò chơi kết thúc
    self.board - trạng thái bàn cờ hiện tại?
    self.territory - trạng thái lãnh thổ hiện tại?
    self.score - điểm số cuối cùng cho cả hai bên
    self.captured - số lượng quân bị bắt từ cả hai bên
  các phương thức:
    place_stone(self,x,y) - cố gắng đặt quân cờ tại vị trí đã cho, cập nhật trạng thái bàn cờ
    legal_move(self,x,y) - trả về true nếu nước đi hợp lệ, ngược lại trả về false
    add_scores(self) - trả về một mảng [x,y] của điểm số cho cả hai người chơi
    get_data(self) - trả về tất cả các dữ liệu liên quan
'''
from cmath import inf
import datetime
from typing import Type
from copy import copy, deepcopy
import random
import numpy as np
class MonteCarloTreeSearchAgent(object):
  '''
    Lớp này nhận mô hình chứa trạng thái trò chơi hiện tại
    nó trả về nước đi tốt nhất được tính toán bởi thuật toán tìm kiếm cây Monte Carlo
  '''
  possibleMoves=[]
  def __init__(self,board):
    self.model=board
    self.n=board.size
    self.parent=None
    self.children=[]
    self.visits=0
    self.scoreSum=0
    self.lastMove=None 
    self.expanded=True
    self.playedMove=False
    self.setPossibleMoves()

  def setPossibleMoves(self):
    if(len(self.possibleMoves)==0):
      self.possibleMoves=[(x,y) for x in range(self.n) for y in range(self.n)]+[(-1,-1)]

  def playMove(self,x,y):
    newBoard=MonteCarloTreeSearchAgent(self.model.copy())
    newBoard.n=self.n
    newBoard.children=[]
    newBoard.parent=self
    newBoard.model.place_stone(y,x)
    newBoard.visits=0
    newBoard.scoreSum=0
    newBoard.expanded=False
    newBoard.lastMove=(y,x)
    return newBoard
      
  def getLastMove(self):
    return self.lastMove

  def legalMoves(self):
    allLegalMoves=[]
    for (x,y) in self.possibleMoves:
      if(self.model.legal_move(y,x)):
        allLegalMoves.append((x,y))
    return allLegalMoves

  def defaultPolicy(self,startNode):
    '''
    Hàm này chơi các nước đi ngẫu nhiên (hợp lệ) cho đến khi trò chơi kết thúc
    Giá trị trả về là đánh giá của vị trí cuối cùng
    '''
    currBoard=startNode.model.copy()
    while not currBoard.game_over:
      next_move=random.choice(self.possibleMoves)
      currBoard.place_stone(next_move[1],next_move[0])
    currBoard.find_territory() 
    return currBoard.getReward()
    
  def expand(self):
    self.expanded=True
    if len(self.children)==0:
      self.children=[self.playMove(x,y) for (x,y) in self.legalMoves()]
    return self


  def score(self,child):
    if(child.visits==0):
      # Giá trị vô cùng cho nút chưa khám phá
      return float(inf) 
    return 1.0*child.scoreSum/child.visits+np.sqrt(2.0*np.log(self.visits)/child.visits)
  
  def avgEval(self,child):
    if(child.visits==0):
      return 0
    return 1.0*child.scoreSum/child.visits
  
  def bestChild(self):
    if len(self.children)==0:
      self.children=[self.playMove(x,y) for (x,y) in self.legalMoves()]
    options=[(self.score(child),child) for child in self.children]
    bestOption=max(options,key=lambda x:x[0])
    return bestOption[1]

  def bestAvgScoreChild(self):
    if len(self.children)==0:
      self.children=[self.playMove(x,y) for (x,y) in self.legalMoves()]
    options=[(self.avgEval(child),child) for child in self.children]
    bestOption=max(options,key=lambda x:x[0])
    return bestOption[1]
  
  def mostVisitedChild(self):
    if len(self.children)==0:
      self.children=[self.playMove(x,y) for (x,y) in self.legalMoves()]
    assert(len(self.children)>0)
    best_children = [self.children[0]]
    for child in self.children[1:]:
      if child.visits > best_children[0].visits or (child.visits == best_children[0].visits and self.avgEval(child) > self.avgEval(best_children[0])):
        best_children = [child]
      elif child.visits == best_children[0].visits and self.avgEval(child) == self.avgEval(best_children[0]):
        best_children.append(child)

    return random.choice(best_children)
  def backup(self,v,delta):
    while v!=None:
      v.visits+=1
      v.scoreSum+=delta
      v=v.parent
    
  def treePolicy(self):
    v=self
    while not v.model.game_over:
      if not v.expanded: #nếu nút chưa được mở rộng
        #bổ sung nút
        return v.expand()
      else:
        v=v.bestChild()
    return v

  def mctsSearch(self):
    '''
    Hàm này thực hiện tìm kiếm cây Monte Carlo
    '''
    move_start_time = datetime.datetime.now()
    move_max_seconds=3
    while datetime.datetime.now()<move_start_time+datetime.timedelta(seconds=move_max_seconds):
      vl=self.treePolicy()
      delta=self.defaultPolicy(vl)
      self.backup(vl,delta)
    #trả nước đi tốt nhất
    for child in self.children:
      print("c:",child.lastMove[1],child.lastMove[0])
      print("visits:",child.visits," scoreSum:",child.scoreSum) #not getting updated at all
    
    return self.mostVisitedChild().getLastMove()



In [4]:
import random

#hàm tạo nước đi ngẫu nhiên hoàn toàn
def get_random_move(model):
    legal_moves = []
    for i in range(model.size):
        for j in range(model.size):
            if model.legal_move(i, j):
                legal_moves.append((i, j))
    return random.choice(legal_moves) if legal_moves else None

print("Enter the board size")
size = int(input())
m = Model(size)

nextPlayerMCTS = True  
consecutive_passes = 0  

while not m.game_over:
    if nextPlayerMCTS:
        agent = MonteCarloTreeSearchAgent(m.copy())
        move = agent.mctsSearch()
    else:
        move = get_random_move(m)
    
    if move is None:
        print("No legal moves available. Game Over!")
        m.game_over = True
        break
    
    if move == (-1, -1):
        consecutive_passes += 1
        if consecutive_passes >= 2:
            m.game_over = True
            break
    else:
        consecutive_passes = 0
        
    print("Player", "Black (MCTS)" if nextPlayerMCTS else "White (Random)", "playing:", move[1], move[0])
    m.place_stone(move[0], move[1])
    print(".......................................")
    m.printBoard()
    
    nextPlayerMCTS = not nextPlayerMCTS

m.find_territory()
scores = m.getFinalScores()
print("Game Over!")
print("Final score - White (Random):", scores[0], "Black (MCTS):", scores[1])

Enter the board size
c: 0 0
visits: 1  scoreSum: -1
c: 0 1
visits: 1  scoreSum: -1
c: 0 2
visits: 1  scoreSum: -1
c: 0 3
visits: 2  scoreSum: 0
c: 0 4
visits: 3  scoreSum: 1
c: 0 5
visits: 1  scoreSum: -1
c: 0 6
visits: 1  scoreSum: -1
c: 0 7
visits: 1  scoreSum: -1
c: 0 8
visits: 1  scoreSum: -1
c: 1 0
visits: 2  scoreSum: 0
c: 1 1
visits: 2  scoreSum: 0
c: 1 2
visits: 2  scoreSum: 0
c: 1 3
visits: 2  scoreSum: 0
c: 1 4
visits: 3  scoreSum: 1
c: 1 5
visits: 1  scoreSum: -1
c: 1 6
visits: 1  scoreSum: -1
c: 1 7
visits: 3  scoreSum: 3
c: 1 8
visits: 3  scoreSum: 1
c: 2 0
visits: 2  scoreSum: 0
c: 2 1
visits: 1  scoreSum: -1
c: 2 2
visits: 1  scoreSum: -1
c: 2 3
visits: 1  scoreSum: -1
c: 2 4
visits: 1  scoreSum: 0
c: 2 5
visits: 2  scoreSum: 0
c: 2 6
visits: 3  scoreSum: 3
c: 2 7
visits: 1  scoreSum: -1
c: 2 8
visits: 1  scoreSum: 0
c: 3 0
visits: 2  scoreSum: 0
c: 3 1
visits: 3  scoreSum: 3
c: 3 2
visits: 2  scoreSum: 0
c: 3 3
visits: 1  scoreSum: -1
c: 3 4
visits: 2  scoreSum: 0
c: 3 